In [8]:
import os
import sys
import gc
import time
import random
import cv2
import glob
import requests
import json
import math
import re
import hashlib
import psutil

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from functools import partial
from collections import Counter
from PIL import Image
from multiprocessing import cpu_count
from tqdm import tqdm
from multiprocessing import Pool
from statistics import median
from joblib import Parallel, delayed

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

from fastai import *
from fastai.vision import *
from fastai.callbacks import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

In [16]:
def crop_image_from_gray(image, tol=8):
    if image.ndim == 2:
        mask = image>told
        return image[np.ix_(mask.any(1),mask.any(0))]
    elif image.ndim== 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mask = gray_image>tol        
        check_shape = image[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return image
        else:
            image1=image[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            image2=image[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            image3=image[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            image = np.stack([image1,image2,image3],axis=-1)
        return image

def save(image_name, path):
    image_size = 300
    image = cv2.imread(path)
    image = crop_image_from_gray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, depth = image.shape
    rate = height / width
    height = int(image_size * rate)
    width = image_size
    image = cv2.resize(image, (height, width))
    image = cv2.addWeighted (image, 4, cv2.GaussianBlur(image , (0,0) , 30) , -4, 128) 
    
    largest_side = np.max((height, width))
    image = cv2.resize(image, (image_size, largest_side))

    height, width, depth = image.shape

    x = width // 2
    y = height // 2
    r = np.amin((x, y))

    circle_image = np.zeros((height, width), np.uint8)
    cv2.circle(circle_image, (x, y), int(r), 1, thickness=-1)
    image = cv2.bitwise_and(image, image, mask=circle_image)
    image = crop_image_from_gray(image)
    cv2.imwrite(os.path.join('..', 'output', 'retina', f'{image_name}.png'), image)

In [17]:
pre_df = pd.read_csv(os.path.join('..', 'input', 'retinopathy-train-2015', 'trainLabels.csv'))
image_name_list = os.listdir(os.path.join('..', 'input', 'retinopathy-train-2015', 'rescaled_train_896'))
image_name_path_list = [os.path.join('..', 'input', 'retinopathy-train-2015', 'rescaled_train_896', f'{image}') for image in image_name_list]
pre_df['path'] = image_name_path_list

In [18]:
base_image_dir = os.path.join('..', 'input')
train_dir = os.path.join(base_image_dir, 'train_images')
train_df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
train_df['path'] = train_df['id_code'].map(lambda x: os.path.join(train_dir, '{}.png'.format(x)))

In [19]:
for (idx, row) in tqdm(train_df.iterrows()):
    save(row['id_code'], row['path'])

3662it [13:51,  3.98it/s]


In [20]:
def save(image_name, path):
    image_size = 300
    image = cv2.imread(path)
    image = crop_image_from_gray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, depth = image.shape
    rate = height / width
    height = int(image_size * rate)
    width = image_size
    image = cv2.resize(image, (height, width))
    image = cv2.addWeighted (image, 4, cv2.GaussianBlur(image , (0,0) , 30) , -4, 128) 
    
    largest_side = np.max((height, width))
    image = cv2.resize(image, (image_size, largest_side))

    height, width, depth = image.shape

    x = width // 2
    y = height // 2
    r = np.amin((x, y))

    circle_image = np.zeros((height, width), np.uint8)
    cv2.circle(circle_image, (x, y), int(r), 1, thickness=-1)
    image = cv2.bitwise_and(image, image, mask=circle_image)
    image = crop_image_from_gray(image)
    cv2.imwrite(os.path.join('..', 'output', 'previous_retina', f'{image_name}.png'), image)

In [22]:
for (idx, row) in tqdm(pre_df.iterrows()):
    save(row['image'], row['path'])

35126it [1:00:32,  8.86it/s]
